In [1]:
import os
import numpy as np
import pandas as pd
import torchvision.transforms as T

from wildlife_datasets import datasets
from analysis import *
from utils import *

c:\Users\lukas\Downloads\GitHub\sides-matching\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = 'MegaDescriptor-L-384'
k = 5
root_datasets = '../wildlife-datasets/data'
img_size = int(model_name.split('-')[-1])

In [3]:
data = [
    (datasets.SarahZelvy, {'daytime': 'all'}),
    (datasets.SeaTurtleIDHeads, {'ignore_no_side': False, 'empty_database': False}),
    (datasets.SeaTurtleIDHeads, {'ignore_no_side': True, 'empty_database': False}),
    (datasets.SeaTurtleIDHeads, {'ignore_no_side': False, 'empty_database': True}),
    (datasets.SeaTurtleIDHeads, {'ignore_no_side': True, 'empty_database': True}),
    (datasets.ZindiTurtleRecall, {'ignore_no_side': False, 'empty_database': False}),
    (datasets.ZindiTurtleRecall, {'ignore_no_side': True, 'empty_database': False}),
    (datasets.ZindiTurtleRecall, {'ignore_no_side': False, 'empty_database': True}),
    (datasets.ZindiTurtleRecall, {'ignore_no_side': True, 'empty_database': True}),
]

for convert_grayscale in [False, True]:
    if convert_grayscale:
        transform = T.Compose([T.Grayscale(3), T.Resize([img_size, img_size]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])    
    else:
        transform = T.Compose([T.Resize([img_size, img_size]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])

    for dataset_class, pars in data:
        dataset_name = dataset_class.__name__
        root = os.path.join(root_datasets, dataset_name)
        file_name = os.path.join('features', f'features_{dataset_name}_{convert_grayscale}_{model_name}.npy')

        d = dataset_class(root)
        dataset = WD(d.df, d.root, transform=transform)
        extractor = get_extractor(model_name='hf-hub:BVRA/'+model_name, batch_size=32, device='cpu')
        features = get_normalized_features(file_name, dataset, extractor)

        if dataset_name == 'SarahZelvy':
            analysis = Analysis_SarahZelvy()
            idx_database, idx_query = analysis.get_split(d.df, daytime=pars['daytime'])
        elif dataset_name in ['SeaTurtleIDHeads', 'ZindiTurtleRecall']:
            if dataset_name == 'SeaTurtleIDHeads':
                analysis = Analysis_SeaTurtleIDHeads()
            else:
                analysis = Analysis_ZindiTurtleRecall()                
            if pars['ignore_no_side']:
                idx_unknown_side = d.df['orientation'].apply(lambda x: x not in analysis.sides.keys())
            else:
                idx_unknown_side = np.zeros(len(d.df), dtype=bool)
            idx_unknown_identity = d.df['identity'] == 'unknown'
            idx_ignore = idx_unknown_side + idx_unknown_identity
            idx_database, idx_query = analysis.get_split(d.df, idx_ignore=idx_ignore, empty_database=pars['empty_database'])
        else:
            raise Exception('Dataset not known')

        if len(idx_database) == 0:
            idx_true, idx_pred = compute_predictions_disjoint(features[idx_query], k=k)
            idx_true = idx_query[idx_true]
            idx_pred = idx_query[idx_pred]
        else:
            idx_true, idx_pred = compute_predictions_closed(features[idx_query], features[idx_database], k=k)
            idx_true = idx_query[idx_true]
            idx_pred = idx_database[idx_pred]

        y_true = dataset.labels_string[idx_true]
        y_pred = dataset.labels_string[idx_pred]
        orientation_true = dataset.metadata['orientation'].to_numpy()[idx_true]
        orientation_pred = dataset.metadata['orientation'].to_numpy()[idx_pred]

        matches = analysis.get_matches(y_true, y_pred, orientation_true, orientation_pred)
        print(dataset_name, convert_grayscale, pars)
        print(f"Database size = {len(idx_database)}. Query size = {len(idx_query)}.")
        print(f"Database individuals = {d.df.iloc[idx_database]['identity'].nunique()}. Query individuals = {d.df.iloc[idx_query]['identity'].nunique()}.")
        display(matches)

SarahZelvy False {'daytime': 'all'}
Database size = 0. Query size = 489.
Database individuals = 0. Query individuals = 60.


,match same side,match diff side,other,wrong match
match k = 1,63.19%,10.43%,0.0%,26.38%
match k = 2,37.63%,20.86%,0.0%,41.51%
match k = 3,28.63%,19.22%,0.0%,52.15%
match k = 4,20.86%,15.34%,0.0%,63.8%
match k = 5,14.93%,13.5%,0.0%,71.57%


SeaTurtleIDHeads False {'ignore_no_side': False, 'empty_database': False}
Database size = 6063. Query size = 1519.
Database individuals = 400. Query individuals = 398.


,match diff = 0,match diff = 1,match diff = 2,match diff = 3,match diff = 4,other,wrong match
match k = 1,61.42%,17.84%,3.36%,2.24%,4.41%,1.65%,9.08%
match k = 2,49.31%,20.08%,5.53%,5.46%,8.03%,1.58%,10.01%
match k = 3,38.05%,21.86%,6.91%,9.15%,11.85%,1.51%,10.66%
match k = 4,34.56%,20.93%,9.08%,9.94%,12.38%,1.65%,11.45%
match k = 5,30.68%,19.75%,8.49%,10.66%,15.87%,1.71%,12.84%


SeaTurtleIDHeads False {'ignore_no_side': True, 'empty_database': False}
Database size = 5967. Query size = 1494.
Database individuals = 400. Query individuals = 397.


...

,match diff = 0,match diff = 1,match diff = 2,other,wrong match
match k = 1,37.24%,11.72%,15.4%,0.0%,35.63%
match k = 2,26.44%,8.97%,15.86%,0.0%,48.74%
match k = 3,22.53%,10.11%,12.41%,0.0%,54.94%
match k = 4,19.31%,8.05%,13.56%,0.0%,59.08%
match k = 5,14.71%,9.89%,11.72%,0.0%,63.68%


ZindiTurtleRecall True {'ignore_no_side': False, 'empty_database': True}
Database size = 0. Query size = 3082.
Database individuals = 0. Query individuals = 2251.


,match diff = 0,match diff = 1,match diff = 2,other,wrong match
match k = 1,1.98%,0.68%,1.23%,5.58%,90.53%
match k = 2,1.04%,0.39%,0.75%,3.73%,94.09%
match k = 3,1.04%,0.26%,0.68%,2.56%,95.46%
match k = 4,0.55%,0.62%,0.49%,2.73%,95.62%
match k = 5,0.65%,0.52%,0.52%,1.75%,96.56%


ZindiTurtleRecall True {'ignore_no_side': True, 'empty_database': True}
Database size = 0. Query size = 435.
Database individuals = 0. Query individuals = 100.


,match diff = 0,match diff = 1,match diff = 2,other,wrong match
match k = 1,18.16%,7.36%,12.41%,0.0%,62.07%
match k = 2,13.79%,7.59%,8.51%,0.0%,70.11%
match k = 3,8.97%,6.67%,5.52%,0.0%,78.85%
match k = 4,6.67%,5.98%,5.98%,0.0%,81.38%
match k = 5,3.91%,5.52%,5.75%,0.0%,84.83%
